In [35]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [36]:
headers = {"Accept-Language": "en-US, en;q=0.5"}

In [37]:
def parse_movie(foobar):
    def get_title(foobar):
        return foobar.h3.a.text
    
    def get_year(foobar):
        return foobar.h3.find('span', class_='lister-item-year').text
    
    def get_runtime(foobar):
        return foobar.find('span', class_='runtime').text if foobar.p.find('span', class_='runtime') else None
    
    def get_imdb_rating(foobar):
        return float(foobar.strong.text)
    
    def get_metascore(foobar):
        return foobar.find('span', class_='metascore').text if foobar.find('span', class_='metascore') else None
    
    def get_votes(foobar):
        nv = foobar.find_all('span', attrs={'name': 'nv'})
        return nv[0].text
    
    def get_gross(foobar):
        nv = foobar.find_all('span', attrs={'name': 'nv'})
        return nv[1].text if len(nv) > 1 else ''  
    
    return {
        'title': get_title(foobar),
        'year': get_year(foobar),
        'runtime': get_runtime(foobar),
        'imdb_rating': get_imdb_rating(foobar),
        'meta_score': get_metascore(foobar),
        'votes': get_votes(foobar),
        'us_gross': get_gross(foobar)}


In [38]:
movies = []

pages = np.arange(1, 1001, 50)

for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&start=" + str(page) + "&ref_=adv_nxt", headers=headers)
    
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    
    movies.extend([parse_movie(foo) for foo in soup.find_all('div', class_='lister-item mode-advanced')])


In [60]:
movies_df = pd.DataFrame(movies)
movies_df.head()

,title,year,runtime,imdb_rating,meta_score,votes,us_gross
0,Soul,(2020),100 min,8.2,83,"97,999",
1,The Godfather,(1972),175 min,9.2,100,"1,608,838",$134.97M
2,The Gentlemen,(2019),113 min,7.8,51,"225,272",
3,Home Alone,(1990),103 min,7.6,63,"483,147",$285.76M
4,Knives Out,(2019),130 min,7.9,82,"438,876",$165.36M


In [61]:
movies_df.loc[:, 'year'] = movies_df['year'].str[-5:-1].astype(int)
movies_df['runtime'] = movies_df['runtime'].str.extract('(\d+)').astype(int)
movies_df['meta_score'] = pd.to_numeric(movies_df['meta_score'].str.extract('(\d+)'))
movies_df['votes'] = movies_df['votes'].str.replace(',', '').astype(int)
movies_df['us_gross'] = movies_df['us_gross'].map(lambda x: x.lstrip('$').rstrip('M'))
movies_df['us_gross'] = pd.to_numeric(movies_df['us_gross'])
# movies_df['us_gross'] = pd.to_numeric(movies['us_gross'], errors='coerce')

movies_df.rename(columns={"runtime": "runtime(mins)", "us_gross": "us_gross(millions)"}, inplace=True)

TypeError: arg must be a list, tuple, 1-d array, or Series

In [59]:
movies_df.head()

,title,year,runtime(mins),imdb_rating,meta_score,votes,us_gross(millions)
0,Soul,2020,100,8.2,83,97999,NaN
1,The Godfather,1972,175,9.2,100,1608838,134.97
2,The Gentlemen,2019,113,7.8,51,225272,NaN
3,Home Alone,1990,103,7.6,63,483147,285.76
4,Knives Out,2019,130,7.9,82,438876,165.36
